# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 2 [10 баллов] до 23:59 22.03.2018. Предсказание цены акции по экономическим новостям


В этом домашнем задании вы попытаетесь предсказать рост цены акции компании Газпром по новостям о компании. Домашнее задание состоит из трех частей:
1. Предварительная обработка текстов и эксплоративный анализ
2. Baseline алгоритм
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



Входные данные:
* Новости о компании "Газпром", начиная с 2010 года
* Стоимость акций компании "Газпром" на ММВБ, начиная с 2010 года
    * цена открытия (Open)
    * цена закрытия (ClosingPrice)
    * максимальная цена за день (DailyHigh)
    * минимальная цена за день (DailyLow) 
    * объем бумаг (VolumePcs)


In [102]:
import numpy as np
import pandas as pd
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold
from scipy.sparse import coo_matrix, hstack
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval

In [2]:
df = pd.read_csv('texts.csv')
df.head()

,date,text
0,09.11.2017,Компания рассчитывает на решение по газовому с...
1,08.11.2017,"Как и предполагал “Ъ”, «Газпром», воспользова..."
2,01.11.2017,Новая редакция американских санкций ставит по...
3,30.10.2017,"Как стало известно “Ъ”, известный на рынке ри..."
4,23.10.2017,"НОВАТЭК, который через пять лет собирается за..."


In [3]:
pr_all = pd.read_csv('gazprom_prices.csv', sep=';')
pr_all.columns = [i.lower() for i in pr_all.columns]
pr_all.head(10)

,date,open,closingprice,dailyhigh,dailylow,volumepcs
0,08.12.2017,"133,43000","132,60000","133,90000","132,00000",16037970
1,07.12.2017,"133,70000","133,02000","133,87000","132,81000",18198430
2,06.12.2017,"133,33000","134,00000","134,29000","132,91000",14641730
3,05.12.2017,"133,48000","133,65000","133,99000","132,78000",12684800
4,04.12.2017,"133,01000","133,77000","134,00000","131,93000",17818980
5,01.12.2017,"132,49000","133,02000","133,32000","131,72000",24755830
6,30.11.2017,"133,00000","132,15000","134,31000","132,00000",40024830
7,29.11.2017,"134,85000","133,55000","134,86000","132,97000",27263040
8,28.11.2017,"133,23000","135,18000","135,18000","132,55000",26663710
9,27.11.2017,"133,69000","133,50000","135,19000","132,80000",27713150


In [4]:
pr_all.dtypes

date            object
open            object
closingprice    object
dailyhigh       object
dailylow        object
volumepcs        int64
dtype: object

Переведем все objects в float.

In [5]:
pr_all['open'] = pr_all['open'].apply(lambda a: str(a).replace(',', '.'))
pr_all['closingprice'] = pr_all['closingprice'].apply(lambda a: str(a).replace(',', '.'))
pr_all['dailyhigh'] = pr_all['dailyhigh'].apply(lambda a: str(a).replace(',', '.'))
pr_all['dailylow'] = pr_all['dailylow'].apply(lambda a: str(a).replace(',', '.'))

pr_all[['open', 'closingprice', 'dailyhigh', 'dailylow']] = pr_all[['open', 'closingprice', 'dailyhigh', 'dailylow']].astype('float')

In [6]:
pr_all.dtypes

date             object
open            float64
closingprice    float64
dailyhigh       float64
dailylow        float64
volumepcs         int64
dtype: object

## Часть 1. Вводная [3 балла]

Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. Ответьте на следующие вопросы:
* Есть ли корреляция между средней длинной текста за день и ценой закрытия?
* Есть ли корреляция между количеством упоминаний Алексея Миллера  и ценой закрытия? Учтите разные варианты написания имени.
* Упоминаний какого газопровода в статьях больше: 
    * "северный поток"
    * "турецкий поток"?
* Кого упоминают чаще:
    * Алексея Миллера
    * Владимира Путина?
* О каких санкциях пишут в статьях?

In [7]:
def beautify_text(s):
    s = re.sub("[^а-яА-Я0-9]", " ", s.lower())
    s = s.replace('\n', '')
    return s

def preprocess_text(text):

    mystem = Mystem()
    sentc = [word_tokenize((beautify_text(i))) for i in text]
    filtered_sentc = [i for i in sentc if i not in stopwords.words('russian')]
    lemmatized_sentc = [[mystem.lemmatize(j)[0] for j in i] for i in sentc]
    
    return lemmatized_sentc

In [8]:
text_pr = preprocess_text(df.text)

In [9]:
df['text_preproc'] = [' '.join(i) for i in text_pr]
df['text_len'] = df.text.apply(lambda a: len(a))
df['miller'] = [' '.join(i).count('алексей миллер') for i in text_pr]

In [10]:
pr_all = pr_all.sort_values(by=['date'])
df = df.sort_values(by=['date'])

In [11]:
gasprom_info = pd.merge(pr_all, df, on=['date'])
gasprom_info.head(10)

,date,open,closingprice,dailyhigh,dailylow,volumepcs,text,text_preproc,text_len,miller
0,01.02.2010,184.74,189.85,190.40,183.50,76298175,"""Газпром"" не исключает в 2010 г. выпуска обли...",газпром не исключать в 2010 г выпуск облигация...,256,0
1,01.02.2011,198.41,204.91,205.00,197.80,87981195,На российском ТВ — вновь дефицит рекламного в...,на российский тв вновь дефицит рекламный время...,586,0
2,01.02.2012,183.00,185.54,186.75,182.60,44145020,Федеральная антимонопольная служба (ФАС) приз...,федеральный антимонопольный служба фас признав...,857,0
3,01.02.2013,142.45,142.41,143.47,141.87,27154010,Правительство выдвинуло 14 кандидатов на 11 м...,правительство выдвигать 14 кандидат на 11 мест...,171,0
4,01.02.2016,136.01,133.90,136.34,132.82,31931470,"""Газпром"" не исключил участия в реализации эк...",газпром не исключать участие в реализация эксп...,1224,0
5,01.02.2017,150.00,149.65,150.38,148.32,20916550,Сегодня исследовательская компания Brand Fina...,сегодня исследовательский компания опубликовыв...,972,0
6,01.03.2012,192.53,194.01,194.27,191.76,31594230,"""Газпром"" скорректирует условия поставок росс...",газпром скорректировать условие поставка росси...,983,2
7,01.03.2017,133.50,133.85,134.99,133.00,38131650,Правление «Газпрома» предложило сохранить ди...,правление газпром предлагать сохранять дивиден...,1167,0
8,01.04.2014,135.90,135.89,136.73,133.84,64684830,Moody's Investors Service поставило рейтинги ...,поставлять рейтинг оао газпром и оао роснефть ...,1398,0
9,01.04.2016,147.02,147.20,147.50,143.93,36517160,"""Газпром-медиа"", управляющий телеканалами ТНТ...",газпром медиа управлять телеканал тнт нтв ради...,789,0


In [12]:
gasprom_info.corr()['text_len']['closingprice']

0.014158394621268122

In [13]:
gasprom_info.corr()['miller']['closingprice']

0.0032372019497298519

In [16]:
print('Северный поток:', sum([' '.join(i).count('северный поток') for i in text_pr]), 
'Турецкий поток:', sum([' '.join(i).count('турецкий поток') for i in text_pr]))

Северный поток: 15 Турецкий поток: 39


In [17]:
print('Алексей Миллер:', sum([' '.join(i).count('алексей миллер') for i in text_pr]), 
'Владимир Путин:', sum([' '.join(i).count('владимир путин') for i in text_pr]))

Алексей Миллер: 125 Владимир Путин: 67


In [18]:
sanc_set = set()
for i in text_pr:
    if 'санкция' in i:
        for j in i:
            if j == 'санкция':
                break
            sanc = j
        if sanc[-2:] in ['ий', 'ой', 'ый']:
            sanc_set.add(sanc)
            
print('Санкции в статьях:', sanc_set)

Санкции в статьях: {'персональный', 'очередной', 'антироссийский', 'западный', 'финансовый', 'экономический', 'штрафной', 'американский', 'международный'}


## Часть 2. Классификационная [3 балла]
Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится цена закрытия.
Для этого:
* бинаризуйте признак "цена закрытия":  новый признак ClosingPrice_bin равен 1, если по сравнению со вчера цена не упала, и 0 – в обратном случае;
* составьте обучающее и тестовое множество: данные до начала 2016 года используются для обучения, данные с 2016 года и позже – для тестирования.

Таким образом, в каждлый момент времени мы знаем: 
* ClosingPrice_bin – бинарый целевой признак
* слова из статей, опубликованных в этот день – объясняющие признаки

В этой части задания вам нужно сделать baseline алгоритм и попытаться его улучшить в следующей части. 

Используйте любой известный вам алгоритм классификации текстов для того, Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure для оценки качества классификации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации.
Если у выбранного вами алгоритма есть гиперпараметры (например, $\alpha$ в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации.

In [19]:
gasprom_info.loc[0, 'closingprice_bin'] = 0
for i in range(1, len(gasprom_info)):
    gasprom_info.loc[i, 'closingprice_bin'] = (1 + (np.sign(gasprom_info.loc[i, 'closingprice'] - gasprom_info.loc[(i-1), 'closingprice']))) / 2

In [20]:
gasprom_info.head()

,date,open,closingprice,dailyhigh,dailylow,volumepcs,text,text_preproc,text_len,miller,closingprice_bin
0,01.02.2010,184.74,189.85,190.40,183.50,76298175,"""Газпром"" не исключает в 2010 г. выпуска обли...",газпром не исключать в 2010 г выпуск облигация...,256,0,0.0
1,01.02.2011,198.41,204.91,205.00,197.80,87981195,На российском ТВ — вновь дефицит рекламного в...,на российский тв вновь дефицит рекламный время...,586,0,1.0
2,01.02.2012,183.00,185.54,186.75,182.60,44145020,Федеральная антимонопольная служба (ФАС) приз...,федеральный антимонопольный служба фас признав...,857,0,0.0
3,01.02.2013,142.45,142.41,143.47,141.87,27154010,Правительство выдвинуло 14 кандидатов на 11 м...,правительство выдвигать 14 кандидат на 11 мест...,171,0,0.0
4,01.02.2016,136.01,133.90,136.34,132.82,31931470,"""Газпром"" не исключил участия в реализации эк...",газпром не исключать участие в реализация эксп...,1224,0,0.0


In [149]:
from sklearn.cross_validation import train_test_split

In [21]:
gasprom_info['date'] = pd.to_datetime(gasprom_info['date'])  

In [88]:
X_train, y_train = gasprom_info[gasprom_info['date'] <= '2016-1-1']['text_preproc'], gasprom_info[gasprom_info['date'] <= '2016-1-1']['closingprice_bin']
X_test, y_test = gasprom_info[gasprom_info['date'] >= '2016-1-1']['text_preproc'], gasprom_info[gasprom_info['date'] >= '2016-1-1']['closingprice_bin']

In [89]:
all_text = pd.concat([X_train, X_test])

In [90]:
word_vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 3),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(X_train)
test_word_features = word_vectorizer.transform(X_test)

char_vectorizer = TfidfVectorizer(
    analyzer='char',
    stop_words='english',
    ngram_range=(1, 3),
    max_features=30000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(X_train)
test_char_features = char_vectorizer.transform(X_test)

In [91]:
train_features = hstack((train_char_features, train_word_features))
test_features = hstack((test_char_features, test_word_features))

In [100]:
shuffle = KFold()
score = cross_val_score(LogisticRegression(), train_features, y_train, cv=shuffle, scoring='accuracy', n_jobs=1)
print('mean cv:', np.mean(score))

mean cv: 0.568239202658


In [110]:
train_word_features[0, 1]

0.0

In [127]:
param_space = {
                'kernel': hp.choice('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
              }

def objective_svc_acc(params):
    model = SVC(kernel=params['kernel'])
    
    shuffle = KFold(n_splits=10)
    score = cross_val_score(model, train_features, y_train, cv=shuffle, scoring='accuracy', n_jobs=1)
    return 1-score.mean()

def objective_svc_f1(params):
    model = SVC(kernel=params['kernel'])
    
    shuffle = KFold()
    score = cross_val_score(model, train_features, y_train, cv=shuffle, scoring='f1', n_jobs=1)
    return 1-score.mean()

trials = Trials()

best = []
print('Fitting model...')
best = fmin(objective_svc_acc,
            param_space,
            algo=tpe.suggest,
            max_evals=10)
print('best acc param:', best)

best = []
print('Fitting model...')
best = fmin(objective_svc_f1,
            param_space,
            algo=tpe.suggest,
            max_evals=10)
print('best f1 param:', best)


Fitting model...
best acc param: {'kernel': 0}
Fitting model...
best f1 param: {'kernel': 1}


In [128]:
cv_score = np.mean(cross_val_score(SVC(kernel='linear'), train_features, y_train, cv=10, scoring='accuracy'))

In [129]:
cv_score

0.57830719841955802

In [134]:
cv_score = np.mean(cross_val_score(SVC(kernel='poly'), train_features, y_train, cv=10, scoring='f1'))

In [135]:
cv_score

0.69325562567848542

## Часть 3. Творческая [4 балла]
Придумайте и попытайтесь сделать еще что-нибудь, чтобы улучшить качество классификации. 
Направления развития:
* Морфологический признаки: 
    * использовать в качестве признаков только существительные или только именованные сущности;
* Модели скрытых тем:
    * использовать в качестве признаков скрытые темы;
    * использовать в качестве признаков динамические скрытые темы 
    пример тут: (https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb)
* Синтаксические признаки:
    * использовать SOV-тройки в качестве признаков
    * кластеризовать SOV-тройки по усредненным эмбеддингам  (обученные word2vec модели можно скачать отсюда: (http://rusvectores.org/ru/models/ или https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md) и использовать только центроиды кластеров в качестве признаков
* что-нибудь еще     

## Сдача домашнего задания

Дедлайн сдачи домашнего задания:  23:59 22.03.2018. Каждый день просрочки дедлайна штрафуется -1 баллом.

Результаты домашнего задания должны быть оформлены в виде отчета в jupyter notebook.
Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание минимума необходимой теории и/или описание используемых инструментов 
* Подробный пошаговый рассказ о проделанной работе
* **Аккуратно** оформленные результаты
* Подробные и внятные ответы на все заданные вопросы 
* Внятные выводы – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Задание выполняется в группе до трех человек. Не забудьте перечислить фамилии всех, кто работал над домашнем задании, в jupyter notebook.  

В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.


При возникновении проблем с выполнением задания обращайтесь с вопросами к преподавателю по семинарским занятиям в вашей группе или у учебным ассистентам.

Учебный ассистент по ДЗ 2: Таисия Глушкова (email: glushkovato@gmail.com, telegram: @glushkovato).


Небрежное оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в отчете в явном виде.

Сдача отчетов осуществляется через систему AnyTask.

